In [1]:
%load_ext lab_black
%cd ..

/home/shim/cev/dl/log-analytics


In [2]:
import argparse
import math
import multiprocessing
import sys
from datetime import datetime
from pathlib import Path
from pprint import pformat

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer
import yaml
from easydict import EasyDict
from pytorch_transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    DistilBertForSequenceClassification,
    DistilBertTokenizer,
    RobertaForSequenceClassification,
    RobertaTokenizer,
)
from sklearn.model_selection import StratifiedKFold
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import (
    AlbertForSequenceClassification,
    AlbertTokenizer,
    DebertaForSequenceClassification,
    DebertaTokenizer,
    SqueezeBertTokenizer,
    SqueezeBertForSequenceClassification,
    XLNetTokenizer,
    XLNetForSequenceClassification,
)

from datasets import load_test_data, load_train_data, load_train_total_data
from utils import SAM, AverageMeter, CustomLogger, FocalLoss, seed_everything

from main import MyTrainer
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from sklearn.metrics import f1_score, classification_report

In [3]:
with open("config/distilbert-base-uncased-ver5-arcface_focal.yaml", "r") as f:
    C = EasyDict(yaml.load(f, yaml.FullLoader))
    C.result_dir = Path(C.result_dir)
    C.dataset.dir = Path(C.dataset.dir)
    seed_everything(C.seed, deterministic=False)

In [4]:
C

{'model': {'name': 'distilbert-base-uncased'},
 'comment': None,
 'result_dir': PosixPath('results/distilbert-base-uncased'),
 'debug': False,
 'seed': 20210425,
 'train': {'SAM': False,
  'folds': [4],
  'checkpoints': [None],
  'loss': {'name': 'arcface',
   'params': {'gamma': 2.0, 's': 45.0, 'm': 0.1, 'crit': 'focal'}},
  'optimizer': {'name': 'AdamW'},
  'finetune': {'do': True, 'step1_epochs': 3, 'step2_epochs': 5},
  'max_epochs': 12,
  'lr': 0.0001,
  'scheduler': {'name': 'ReduceLROnPlateau',
   'params': {'factor': 0.5, 'patience': 3, 'verbose': True}}},
 'dataset': {'dir': PosixPath('data/ori'),
  'ver': 5,
  'batch_size': 35,
  'num_workers': 8}}

In [5]:
trainer = MyTrainer(C, 1)

In [6]:
tds = trainer.tdl.dataset

In [7]:
type(tds)

datasets.dataset_ver5.MyDatasetVer5

In [8]:
tds[0]

(0,
 tensor([  101,  3204,  2154,  2051,  2334, 15006,  2102, 11382, 19445,  1024,
          1063,  1000,  2828,  1000,  1024,  1000,  7561,  1000,  1010,  1000,
          1030,  2335, 15464,  2361,  1000,  1024,  1000,  3058,  1056,  2051,
          1062,  1000,  1010,  1000, 22073,  1000,  1024,  1031,  1000,  5432,
          1000,  1010,  1000, 26319,  1011,  3074,  1000,  1033,  1010,  1000,
         14255,  2094,  1000,  1024,  4185, 27814,  1010,  1000,  2504,  1000,
          1024,  1000,  7561,  1000,  1010,  1000,  7561,  1000,  1024,  1063,
          1000,  4471,  1000,  1024,  1000,  2053,  2542,  7264,  1000,  1010,
          1000,  2171,  1000,  1024,  1000,  7561,  1000,  1010,  1000,  9991,
          1000,  1024,  1000,  7561,  1024,  2053,  2542,  7264,  2012,  4604,
          2890,  4160, 24415,  8663,  2638,  7542,  1006,  1013,  2149,  2099,
          1013,  3745,  1013, 11382, 19445,  1013, 13045,  1035, 14184,  1013,
         21274, 17310, 11140,  1013,  5034,  227

In [10]:
trainer.tokenizer.decode(tds[0][1].tolist())

[' month day time localhost kibana : { " type " : " error ", " @ timestamp " : " date t time z ", " tags " : [ " warning ", " stats - collection " ], " pid " : 6458, " level " : " error ", " error " : { " message " : " no living connections ", " name " : " error ", " stack " : " error : no living connections at sendreqwithconnection ( / usr / share / kibana / node _ modules / elasticsearch / src / lib / transport. js : 226 : 15 ) at next ( / usr / share / kibana / node _ modules / elasticsearch / src / lib / connection _ pool. js : 214 : 7 ) at process. _ tickcallback ( internal / process / next _ tick. js : 61 : 11 ) " }, " message " : " no living connections " } ',
 ' [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA